In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import data_loader
import datetime
import pickle
import timeit
import time
import os
from utils import *
tf.keras.backend.set_floatx('float32')
os.makedirs('./cgan_samples', exist_ok=True)

# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[3],'GPU')

In [ ]:
def sample_generation(no_samples,temp):
    n_zrand = 12   # 6 for 4x4,6x6,8x8 and 12  for 16x16
    n_z     = 6
    zrand   = tf.random.normal([no_samples,n_zrand],mean=0.0,stddev=1.0,dtype=tf.dtypes.float32)
    tz      = tf.cast(tf.repeat(temp,no_samples*n_z),dtype=tf.float32)
    tz      = tf.reshape(tz,[no_samples,n_z])
    z       = tf.concat([zrand,tz],axis=-1)
    samples = generator(z)
    return samples

In [ ]:
def samples_generation_for_all_temp(no_samples,temp_array):
    samples = []
    for i in temp_array:
        samples.append(sample_generation(no_samples,i))
    samples = tf.concat(samples,axis=0)
    return samples
        

In [ ]:
EPOCH = 50
BATCH_SIZE = 256
noise_dim = 18
num_classes = 32
(l,l) = (16,16) # lattice Size
n_z = 6 #No.of temp.Variables
n_zrand = 12 #No. of Noise Variables


In [ ]:
xy_data = np.float32(data_loader.load_data_mh_generated('./data/data/xy_16x16_32_lattices_set1.pkl'))
trainset  = []
   
for i in range(num_classes):
    trainset.append(xy_data[10000*i:10000*i+5000])

trainset = np.reshape(np.array(trainset),(-1,l,l,1))

temp_val = np.linspace(0.05,2.05,num_classes)
Temp = np.float32(np.repeat(temp_val,5000))
T = np.repeat(Temp,l*l).reshape(-1,l,l,1)
T_z = np.repeat(Temp,n_z).reshape(-1,n_z)

training_dataset = tf.data.Dataset.from_tensor_slices((trainset,T,T_z))
training_dataset = training_dataset.shuffle(buffer_size = 1024).batch(BATCH_SIZE)

In [ ]:
# def make_generator(): #### 8x8 lattices
#     z = keras.Input(shape=(12,))
#     hid_x1 = layers.Dense(32,activation='tanh',name='hidden_layer_1')(z)
#     hid_x2 = layers.Dense(128,activation='tanh',name='hidden_layer_2')(hid_x1)
#     reshape_x3 = layers.Reshape((4,4,8))(hid_x2)
#     conv1 = layers.Conv2DTranspose(filters=20,kernel_size=(3,3),strides=1,padding='valid',activation='tanh')(reshape_x3)
#     conv_mean = layers.Conv2DTranspose(filters=1,kernel_size=(3,3),strides=1,padding='valid')(conv1)
#     conv_scale = layers.Conv2DTranspose(filters=1,kernel_size=(3,3),strides=1,padding='valid')(conv1)
#     eps = tf.random.normal(shape = tf.shape(conv_mean),mean =0.0 ,stddev = 1.0, dtype = tf.float32 )
#     sample = conv_mean + eps*tf.sqrt(tf.exp(5*conv_scale))
#     model = keras.Model(inputs=z,outputs=sample,name='Generator_model')
#     model.summary()
#     return model

def make_generator(): #### 16x16 lattices
    z = keras.Input(shape=(18,))
    hid_x1 = layers.Dense(32,activation='tanh',name='hidden_layer_1')(z)
    hid_x2 = layers.Dense(128,activation='tanh',name='hidden_layer_2')(hid_x1)
    reshape_x3 = layers.Reshape((8,8,2))(hid_x2)
    conv1 = layers.Conv2DTranspose(filters=8,kernel_size=(3,3),strides=1,padding='valid',activation='tanh')(reshape_x3)
    conv2 = layers.Conv2DTranspose(filters=16,kernel_size=(3,3),strides=1,padding='valid',activation='tanh')(conv1)
    conv3 = layers.Conv2DTranspose(filters=32,kernel_size=(3,3),strides=1,padding='valid',activation='tanh')(conv2)

    conv_mean = layers.Conv2DTranspose(filters=1,kernel_size=(3,3),strides=1,padding='valid')(conv3)
    conv_scale = layers.Conv2DTranspose(filters=1,kernel_size=(3,3),strides=1,padding='valid')(conv3)
    eps = tf.random.normal(shape = tf.shape(conv_mean),mean =0.0 ,stddev = 1.0, dtype = tf.float32 )
    sample = conv_mean + eps*tf.sqrt(tf.exp(5*conv_scale))
    model = keras.Model(inputs=z,outputs=sample,name='Generator_model')
    model.summary()
    return model


# def make_generator(): #### 6x6 lattices
#     z = keras.Input(shape=(12,))
#     hid_x1 = layers.Dense(32,activation='tanh',name='hidden_layer_1')(z)
#     hid_x2 = layers.Dense(128,activation='tanh',name='hidden_layer_2')(hid_x1)
#     hid_x3 = layers.Reshape((4,4,8))(hid_x2)
#     conv_mean = layers.Conv2DTranspose(filters=1,kernel_size=(3,3),strides=1,padding='valid')(hid_x3)
#     conv_scale = layers.Conv2DTranspose(filters=1,kernel_size=(3,3),strides=1,padding='valid')(hid_x3)
#     eps = tf.random.normal(shape = tf.shape(conv_mean),mean =0.0 ,stddev = 1.0, dtype = tf.float32 )
#     sample = conv_mean + eps*tf.sqrt(tf.exp(5*conv_scale))
#     model = keras.Model(inputs=z,outputs=sample,name='Generator_model')
#     model.summary()
#     return model 


# def make_generator(): #### 4x4 lattices
#     z = keras.Input(shape=(12,))
#     hid_x1 = layers.Dense(32,activation='tanh',name='hidden_layer_1')(z)
#     hid_x2 = layers.Dense(128,activation='tanh',name='hidden_layer_2')(hid_x1)
#     hid_x3 = layers.Reshape((4,4,8))(hid_x2)
#     hid_x4 = layers.Conv2DTranspose(filters = 8,kernel_size=(3,3),strides=1,padding='valid')(hid_x3)
#     conv_mean = layers.Conv2D(filters=1,kernel_size=(3,3),strides=1,padding='valid')(hid_x4)
#     conv_scale = layers.Conv2D(filters=1,kernel_size=(3,3),strides=1,padding='valid')(hid_x4)
#     eps = tf.random.normal(shape = tf.shape(conv_mean),mean =0.0 ,stddev = 1.0, dtype = tf.float32 )
#     sample = conv_mean + eps*tf.sqrt(tf.exp(5*conv_scale))
#     model = keras.Model(inputs=z,outputs=sample,name='Generator_model')
#     model.summary()
#     return model 

In [ ]:
# def make_discriminator(): # 8x8 lattices
#     input = keras.Input(shape=(l,l,2))
#     hid_y1 = layers.Conv2D(filters=10,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer1')(periodic_padding(input,1))
#     hid_y2 = layers.MaxPooling2D(pool_size=(2,2),padding='valid',name='layer2')(hid_y1)
#     hid_y3 = layers.Conv2D(filters=32,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer3')(hid_y2)
#     reshape_y4 = layers.Reshape((2*2*32,))(hid_y3)
#     FC_y5 = layers.Dense(32,activation='tanh',name='Dense_layer5')(reshape_y4)
#     FC_y6 = layers.Dense(10,activation='tanh',name='Dense_layer6')(FC_y5)
#     output = layers.Dense(1,name='Disc_output')(FC_y6)
#     model = keras.Model(inputs=input,outputs= output,name='Discriminator_Model')
#     model.summary()
#     return model


def make_discriminator(): # 16X16 lattices
    input = keras.Input(shape=(l,l,2))
    hid_y1 = layers.Conv2D(filters=10,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer1')(periodic_padding(input,1))
    hid_y2 = layers.MaxPooling2D(pool_size=(2,2),padding='valid',name='layer2')(hid_y1)
    hid_y3 = layers.Conv2D(filters=16,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer3')(periodic_padding(hid_y2,1))
    hid_y4 = layers.MaxPooling2D(pool_size=(2,2),padding='valid',name='layer4')(hid_y3)
    hid_y5 = layers.Conv2D(filters=32,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer5')(hid_y4)

    reshape_y4 = layers.Reshape((2*2*32,))(hid_y5)
    FC_y5 = layers.Dense(32,activation='tanh',name='Dense_layer5')(reshape_y4)
    FC_y6 = layers.Dense(20,activation='tanh',name='Dense_layer6')(FC_y5)
    output = layers.Dense(1,name='Disc_output')(FC_y6)
    model = keras.Model(inputs=input,outputs= output,name='Discriminator_Model')
    model.summary()
    return model

# def make_discriminator(): # 6x6 lattices
#     input = keras.Input(shape=(l,l,2))
#     hid_y1 = layers.Conv2D(filters=10,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer1')(periodic_padding(input,1))
#     hid_y2 = layers.Conv2D(filters=16,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer2')(hid_y1)
#     hid_y3 = layers.Conv2D(filters=32,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer3')(hid_y2)

#     reshape_y4 = layers.Reshape((2*2*32,))(hid_y3)
#     FC_y5 = layers.Dense(32,activation='tanh',name='Dense_layer5')(reshape_y4)
#     FC_y6 = layers.Dense(10,activation='tanh',name='Dense_layer6')(FC_y5)
#     output = layers.Dense(1,name='Disc_output')(FC_y6)
#     model = keras.Model(inputs=input,outputs= output,name='Discriminator_Model')
#     model.summary()
#     return model

# def make_discriminator(): # 4x4 lattices
#     input = keras.Input(shape=(l,l,2))
#     hid_y1 = layers.Conv2D(filters=10,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer1')(periodic_padding(input,1))
#     hid_y2 = layers.Conv2D(filters=64,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer2')(hid_y1)
    
#     reshape_y4 = layers.Reshape((2*2*32,))(hid_y2)
#     FC_y5 = layers.Dense(32,activation='tanh',name='Dense_layer5')(reshape_y4)
#     FC_y6 = layers.Dense(10,activation='tanh',name='Dense_layer6')(FC_y5)
#     output = layers.Dense(1,name='Disc_output')(FC_y6)
#     model = keras.Model(inputs=input,outputs= output,name='Discriminator_Model')
#     model.summary()
#     return model


    

In [ ]:
generator = make_generator()
discriminator = make_discriminator()
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output,fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)



In [ ]:
boundaries = [5000*1, 5000*2, 5000*3, 5000*4, 5000*5, 5000*6, 5000*7, 5000*8, 5000*9, 5000*10]
 
gen_lr = 5.e-7
dis_lr = 15.e-7
decay  = 0.95
values_gen = [gen_lr, gen_lr*decay, gen_lr*(decay)**2, gen_lr*(decay)**3, gen_lr*(decay)**4,
              gen_lr*(decay)**5, gen_lr*(decay)**6, gen_lr*(decay)**7, gen_lr*(decay)**8, 
              gen_lr*(decay)**9, gen_lr*(decay)**10]
values_dis = [dis_lr, dis_lr*decay, dis_lr*(decay)**2, dis_lr*(decay)**3, dis_lr*(decay)**4,
              dis_lr*(decay)**5, dis_lr*(decay)**6, dis_lr*(decay)**7, dis_lr*(decay)**8, 
              dis_lr*(decay)**9, dis_lr*(decay)**10]
learning_rate_fn_gen = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values_gen)
learning_rate_fn_dis = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values_dis)

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn_gen)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn_dis)
train_G_loss = tf.keras.metrics.Mean('train_G_loss', dtype=tf.float32)
train_D_loss = tf.keras.metrics.Mean('train_D_loss', dtype=tf.float32)


In [ ]:
@tf.function
def train_step(x,y,tz):
    zrand = tf.cast(np.random.normal(size=[tz.shape[0],n_zrand]),dtype = tf.float32)
    z_rand = tf.convert_to_tensor(zrand)
    noise = tf.concat([z_rand,tz],axis=1)
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_data = tf.cast(generator(noise,training = True),dtype=tf.float32)
        input1 = tf.concat([x,y],axis = -1)
        input2 = tf.concat([gen_data,y],axis = -1)
        real_output = discriminator(input1,training=True)
        fake_output = discriminator(input2,training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output,fake_output)
        
    gradients_of_generator = gen_tape.gradient(gen_loss,generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator,generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator,discriminator.trainable_variables))
    train_G_loss(gen_loss)
    train_D_loss(disc_loss)


In [ ]:
checkpoint_directory = './cgan_samples/16x16/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_directory, "ckpt-5")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer= discriminator_optimizer,
                                 generator = generator,
                                 discriminator = discriminator)

In [ ]:
checkpoint.restore(checkpoint_prefix)

In [ ]:
start = time.time()
for epoch in range(10):
    
    for step,(x,y,tz) in enumerate(training_dataset):
        train_step(x,y,tz)

    template = 'Epoch {}, Gen_Loss: {}, Disc_Loss: {}'
    print (template.format(epoch+1,train_G_loss.result(),train_D_loss.result()))

    # Reset metrics every epoch
    train_G_loss.reset_states()
    train_D_loss.reset_states() 
  
stop = time.time()
print('Time: ', stop - start)
            

In [ ]:
#Plotting Comparison Plots for different epochs
lattices1=[]
index_set = np.arange(num_classes)
for i in index_set:
    lattices1.append(xy_data[10000*i+5000:10000*i+6000])
lattices1=np.array(lattices1)
lattices1=tf.reshape(lattices1,(-1,l,l,1))



In [ ]:
checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
samples = samples_generation_for_all_temp(1000,temp_val)

In [ ]:
comparison_plot(lattices1,samples,1000,0.05,2.05,num_classes,J=1,K=0,name='./cgan_samples/16x16/comparison_plot_ckpt_5_6')

In [ ]:
evaluation_metrics(lattices1,samples,0.05,2.05,num_classes,J=1,K=0)

In [ ]:
output = open('./cgan_samples/16x16/lattices_ckpt_5_6.pkl', 'wb')
pickle.dump(samples, output)
output.close()